In [57]:
import pandas as pd
import seaborn as sns
import numpy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
#from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold


def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def confusionMatrix(y_true, y_pred):
    return (tp(y_true, y_pred), tn(y_true, y_pred), fp(y_true, y_pred), fn(y_true, y_pred))

def predict_logistic(filename):
    
    dataset = pd.read_csv(filename)
    header = list(dataset.columns.values)
    
    logistic = LogisticRegression()
    
    training_features = header[0:-1]
    target = 'is_mh_sw'
    
    pos = dataset.loc[dataset[target] == 1]
    neg = dataset.loc[dataset[target] == 0]

    #test = pd.concat([pos.sample(n=round(len(pos)*0.2)),neg.sample(n=round(len(neg)*0.2))])
    test = pd.concat([pos.sample(n=round(len(pos)*0.1)),neg.sample(n=round(len(neg)*0.1))])
    index = list(test.index.values)
    train = dataset.drop(index)

    train_x = train[training_features]
    test_x = test[training_features]
    train_y = train[target]
    test_y = test[target]
    
    skf = StratifiedKFold(n_splits=10)
    skf.get_n_splits(dataset[training_features], dataset[target])
    #train_x, test_x, train_y, test_y = train_test_split(dataset[training_features], dataset[target], train_size=0.9)
    #for train_index, test_index in skf.split(X, y):
    result = []
    y_true = []
    for train_index, test_index in skf.split(dataset[training_features], dataset[target]):
        X_train, X_test = dataset.loc[train_index, training_features], dataset.loc[test_index, training_features]
        Y_train, Y_test = dataset.loc[train_index, target], dataset.loc[test_index, target]
        logistic.fit(X_train, Y_train)
        predicted = logistic.predict(X_test)
        y_true.append(Y_test)
        result.append(predicted)
        
    accuracy = []
    precision = []
    recall = []
    f1 = []
    tp1 = []
    tn1 = []
    fp1 = []
    fn1 = []
    
    for i in range(1,10):
        accuracy.append(metrics.accuracy_score(y_true[i], result[i]))
        precision.append(metrics.precision_score(y_true[i], result[i]))
        recall.append(metrics.recall_score(y_true[i], result[i]))
        f1.append(metrics.f1_score(y_true[i], result[i]))
        tp1.append(tp(y_true[i], result[i]))
        tn1.append(tn(y_true[i], result[i]))
        fp1.append(fp(y_true[i], result[i]))
        fn1.append(fn(y_true[i], result[i]))
        
#     print("Accuracy: ")
#     print(accuracy)
#     print(numpy.mean(accuracy))
#     print(numpy.std(accuracy))  
    print("Precision: ")
    print(precision)
    print(numpy.mean(precision))
    print(numpy.std(precision))    
    print("Recall: ")
    print(recall)
    print(numpy.mean(recall))
    print(numpy.std(recall))
    print("F-1: ")
    print(f1)
    print(numpy.mean(f1))
    print(numpy.std(f1))
    print("TP: ")
    print(tp1)
    print("TN: ")
    print(tn1)
    print("FP: ")
    print(fp1)
    print("FN: ")
    print(fn1)
#     print("Number of Observations :: ", len(dataset))
#     print("train_x size :: ", train_x.shape)
#     print("train_y size :: ", train_y.shape)
#     print("test_x size :: ", test_x.shape)
#     print("test_y size :: ", test_y.shape)
#     print("Accuracy score: ", metrics.accuracy_score(test_y, predicted))
#     print("ROC_AUC socre: ", metrics.roc_auc_score(test_y, probs[:, 1]))
#     tn, fp, fn, tp = metrics.confusion_matrix(test_y, predicted).ravel()
#     print("tp, tn, fp, fn: ", tp, tn, fp, fn)
#     print(metrics.confusion_matrix(test_y, predicted))
#     print(metrics.classification_report(test_y, predicted))
    
def predict_nb(filename):
    dataset = pd.read_csv(filename)
    header = list(dataset.columns.values)
    
    gnb = GaussianNB()
    
    training_features = header[0:-1]
    target = 'is_mh_sw'
    
    pos = dataset.loc[dataset[target] == 1]
    neg = dataset.loc[dataset[target] == 0]

    #test = pd.concat([pos.sample(n=round(len(pos)*0.2)),neg.sample(n=round(len(neg)*0.2))])
    test = pd.concat([pos.sample(n=round(len(pos)*0.1)),neg.sample(n=round(len(neg)*0.1))])
    index = list(test.index.values)
    train = dataset.drop(index)

    train_x = train[training_features]
    test_x = test[training_features]
    train_y = train[target]
    test_y = test[target]
    
    #train_x, test_x, train_y, test_y = train_test_split(dataset[training_features], dataset[target], train_size=0.9)

    gnb.fit(train_x, train_y)
    gnb_predicted = gnb.predict(test_x)
    gnb_probs = gnb.predict_proba(test_x)

    #print("Number of Observations :: ", len(dataset))
    #print("train_x size :: ", train_x.shape)
    #print("train_y size :: ", train_y.shape)
    #print("test_x size :: ", test_x.shape)
    #print("test_y size :: ", test_y.shape)
    print("Gaussian NB accuracy score: ", metrics.accuracy_score(test_y, gnb_predicted))
    print("Gaussian NB ROC_AUC socre: ", metrics.roc_auc_score(test_y, gnb_probs[:, 1]))
    tn, fp, fn, tp = metrics.confusion_matrix(test_y, gnb_predicted).ravel()
    print("tp, tn, fp, fn: ", tp, tn, fp, fn)
    print(metrics.confusion_matrix(test_y, gnb_predicted))
    print(metrics.classification_report(test_y, gnb_predicted))

def predict_svm(filename):
    dataset = pd.read_csv(filename)
    header = list(dataset.columns.values)
    
    clf = svm.SVC()
    
    training_features = header[0:-1]
    target = 'is_mh_sw'
    
    pos = dataset.loc[dataset[target] == 1]
    neg = dataset.loc[dataset[target] == 0]

    #test = pd.concat([pos.sample(n=round(len(pos)*0.2)),neg.sample(n=round(len(neg)*0.2))])
    test = pd.concat([pos.sample(n=round(len(pos)*0.1)),neg.sample(n=round(len(neg)*0.1))])
    index = list(test.index.values)
    train = dataset.drop(index)

    train_x = train[training_features]
    test_x = test[training_features]
    train_y = train[target]
    test_y = test[target]
    
    #train_x, test_x, train_y, test_y = train_test_split(dataset[training_features], dataset[target], train_size=0.9)

    clf.fit(train_x, train_y)
    clf_predicted = clf.predict(test_x)
    #clf_probs = clf.predict_proba(test_x)

    #print("Number of Observations :: ", len(dataset))
    #print("train_x size :: ", train_x.shape)
    #print("train_y size :: ", train_y.shape)
    #print("test_x size :: ", test_x.shape)
    #print("test_y size :: ", test_y.shape)
    print("SVM accuracy score: ", metrics.accuracy_score(test_y, clf_predicted))
    #print("SVM ROC_AUC socre: ", metrics.roc_auc_score(test_y, clf_probs[:, 1]))
    tn, fp, fn, tp = metrics.confusion_matrix(test_y, clf_predicted).ravel()
    print("tp, tn, fp, fn: ", tp, tn, fp, fn)
    print(metrics.confusion_matrix(test_y, clf_predicted))
    print(metrics.classification_report(test_y, clf_predicted))    

In [58]:
# predict_logistic("dataset_full.csv")

In [59]:
def logistic_cv(filename):
    
    dataset = pd.read_csv(filename)
    header = list(dataset.columns.values)
    
    logistic = LogisticRegression()
    
    training_features = header[0:-1]
    target = 'is_mh_sw'
    
#     accuracy = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(metrics.accuracy_score))    
#     precision = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(metrics.precision_score))
#     recall = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(metrics.recall_score))
#     f1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(metrics.f1_score))
#     tp1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(tp))
#     tn1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(tn))
#     fp1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(fp))
#     fn1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=5,
#                              scoring = make_scorer(fn))

    
    accuracy = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.accuracy_score))    
    precision = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.precision_score))
    recall = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.recall_score))
    f1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.f1_score))
    tp1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(tp))
    tn1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(tn))
    fp1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(fp))
    fn1 = cross_val_score(logistic, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(fn))

    print("Accuracy: ")
    print(accuracy)
    print(numpy.mean(accuracy))
    #print(numpy.std(accuracy))  
    print("Precision: ")
    print(precision)
    print(numpy.mean(precision))
    #print(numpy.std(precision))    
    print("Recall: ")
    print(recall)
    print(numpy.mean(recall))
    #print(numpy.std(recall))
    print("F-1: ")
    print(f1)
    print(numpy.mean(f1))
    print(numpy.std(f1))
#     print("TP: ")
#     print(tp1)
#     print("TN: ")
#     print(tn1)
#     print("FP: ")
#     print(fp1)
#     print("FN: ")
#     print(fn1)

#     print("Precision: ")
#     print(numpy.mean(precision))
#     print("Recall: ")
#     print(numpy.mean(recall))
#     print("F-1: ")
#     print(numpy.mean(f1))
    
def nb_cv(filename):
    
    dataset = pd.read_csv(filename)
    header = list(dataset.columns.values)
    
    nb = GaussianNB()
    
    training_features = header[0:-1]
    target = 'is_mh_sw'        
    
    accuracy = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.accuracy_score))    
    precision = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.precision_score))
    recall = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.recall_score))
    f1 = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.f1_score))
    tp1 = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(tp))
    tn1 = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(tn))
    fp1 = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(fp))
    fn1 = cross_val_score(nb, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(fn))
    
    print("Accuracy: ")
    print(accuracy)
    print(numpy.mean(accuracy))
    #print(numpy.std(accuracy))  
    print("Precision: ")
    print(precision)
    print(numpy.mean(precision))
    #print(numpy.std(precision))    
    print("Recall: ")
    print(recall)
    print(numpy.mean(recall))
    #print(numpy.std(recall))
    print("F-1: ")
    print(f1)
    print(numpy.mean(f1))
    #print(numpy.std(f1))
#     print("TP: ")
#     print(tp1)
#     print("TN: ")
#     print(tn1)
#     print("FP: ")
#     print(fp1)
#     print("FN: ")
#     print(fn1)

#     print("Precision: ")
#     print(numpy.mean(precision))
#     print("Recall: ")
#     print(numpy.mean(recall))
#     print("F-1: ")
#     print(numpy.mean(f1))
    
def svm_cv(filename):
    
    dataset = pd.read_csv(filename)
    header = list(dataset.columns.values)
    
    clf = svm.SVC()
    
    training_features = header[0:-1]
    target = 'is_mh_sw'        
        
    accuracy = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.accuracy_score))    
    precision = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.precision_score))
    recall = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.recall_score))
    f1 = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(metrics.f1_score))
    tp1 = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(tp))
    tn1 = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(tn))
    fp1 = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(fp))
    fn1 = cross_val_score(clf, dataset[training_features], dataset[target], cv=10,
                             scoring = make_scorer(fn))
    
    print("Accuracy: ")
    print(accuracy)
    print(numpy.mean(accuracy))
    #print(numpy.std(accuracy))  
    print("Precision: ")
    print(precision)
    print(numpy.mean(precision))
    #print(numpy.std(precision))    
    print("Recall: ")
    print(recall)
    print(numpy.mean(recall))
    #print(numpy.std(recall))
    print("F-1: ")
    print(f1)
    print(numpy.mean(f1))
    print(numpy.std(f1))
#     print("TP: ")
#     print(tp1)
#     print("TN: ")
#     print(tn1)
#     print("FP: ")
#     print(fp1)
#     print("FN: ")
#     print(fn1)
    
#     print("Precision: ")
#     print(numpy.mean(precision))
#     print("Recall: ")
#     print(numpy.mean(recall))
#     print("F-1: ")
#     print(numpy.mean(f1))

In [60]:
# logistic_cv("dataset_interpersonal.csv")
# logistic_cv("dataset_metadata.csv")
# logistic_cv("dataset_sentiment.csv")
# logistic_cv("dataset_linguistic.csv")
# logistic_cv("dataset_content.csv")
logistic_cv("dataset_full.csv")

Accuracy: 
[ 0.71276596  0.67021277  0.79787234  0.71276596  0.67021277  0.73913043
  0.68478261  0.76086957  0.83695652  0.70652174]
0.72920906568
Precision: 
[ 0.77777778  0.73529412  0.85        0.76315789  0.68181818  0.76190476
  0.69767442  0.8         0.87804878  0.71111111]
0.765678704409
Recall: 
[ 0.59574468  0.53191489  0.72340426  0.61702128  0.63829787  0.69565217
  0.65217391  0.69565217  0.7826087   0.69565217]
0.662812210916
F-1: 
[ 0.6746988   0.61728395  0.7816092   0.68235294  0.65934066  0.72727273
  0.6741573   0.74418605  0.82758621  0.7032967 ]
0.709178452907
0.0590910137217


In [52]:
# nb_cv("dataset_interpersonal.csv")
# nb_cv("dataset_metadata.csv")
# nb_cv("dataset_sentiment.csv")
# nb_cv("dataset_linguistic.csv")
# nb_cv("dataset_content.csv")
# nb_cv("dataset_full.csv")

In [53]:
# svm_cv("dataset_interpersonal.csv")
# svm_cv("dataset_metadata.csv")
# svm_cv("dataset_sentiment.csv")
# svm_cv("dataset_linguistic.csv")
# svm_cv("dataset_content.csv")
# svm_cv("dataset_full.csv")

In [54]:
logistic_cv("dataset_interpersonal_2.csv")
logistic_cv("dataset_metadata_2.csv")
logistic_cv("dataset_sentiment_2.csv")
logistic_cv("dataset_linguistic_2.csv")
logistic_cv("dataset_content_2.csv")
logistic_cv("dataset_full_2.csv")

C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2475 2476 2476 2476 2472 2474 2476 2476 2475 2476]
FP: 
[2 0 0 0 4 2 0 0 1 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

...

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 1 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2475 2475 2476 2475 2476 2475 2475]
FP: 
[0 0 0 1 1 0 1 0 1 1]
FN: 
[47 46 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[1 0 0 0 0 0 0 0 0 0]
TN: 
[2473 2476 2476 2475 2475 2476 2476 2476 2476 2476]
FP: 
[4 0 0 1 1 0 0 0 0 0]
FN: 
[46 47 47 47 47 46 46 46 46 46]


In [ ]:
nb_cv("dataset_interpersonal_2.csv")
nb_cv("dataset_metadata_2.csv")
nb_cv("dataset_sentiment_2.csv")
nb_cv("dataset_linguistic_2.csv")
nb_cv("dataset_content_2.csv")
nb_cv("dataset_full_2.csv")

C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]
TP: 
[2 3 5 0 4 4 3 3 1 3]
TN: 
[2427 2440 2433 2436 2420 2434 2433 2447 2418 2410]
FP: 
[50 36 43 40 56 42 43 29 58 66]
FN: 
[45 44 42 47 43 42 43 43 45 43]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]
TP: 
[23 31 28 28 28 20 27 29 30 21]
TN: 
[1792 1821 1848 1846 1873 1834 1838 1669 1785 1818]
FP: 
[685 655 628 630 603 642 638 807 691 658]
FN: 
[24 16 19 19 19 26 19 17 16 25]
TP: 
[24 29 22 27 27 20 27 25 28 20]
TN: 
[1891 1912 1913 1923 1945 1920 1899 1922 1864 1888]
FP: 
[586 564 563 553 531 556 577 554 612 588]
FN: 
[23 18 25 20 20 26 19 21 18 26]


In [ ]:
svm_cv("dataset_interpersonal_2.csv")
svm_cv("dataset_metadata_2.csv")
svm_cv("dataset_sentiment_2.csv")
svm_cv("dataset_linguistic_2.csv")
svm_cv("dataset_content_2.csv")
svm_cv("dataset_full_2.csv")

C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

...

C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp

TP: 
[0 0 0 0 0 0 0 0 0 0]
TN: 
[2477 2476 2476 2476 2476 2476 2476 2476 2476 2476]
FP: 
[0 0 0 0 0 0 0 0 0 0]
FN: 
[47 47 47 47 47 46 46 46 46 46]


C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phuc\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samp